In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
fpath = f'scrubbed.csv'
colnames = [
        'datetime',
        'city',
        'state',
        'country',
        'shape',
        'duration',
        'duration (hours/min)',
        'comments',
        'date posted',
        'latitude',
        'longitude',
        'empty'
    ]

with open(fpath,'r') as file:
    df = pd.read_csv(file, names=colnames, skiprows=1)

to_drop = [
    'empty', 
    'duration (hours/min)', 
    'date posted'
]
df.drop(labels=to_drop, axis=1, inplace = True)
df.head()

In [3]:
for row, cols in df.iterrows():
    if '24:00' in cols['datetime']:
        df.loc[row, 'datetime'] = cols['datetime'].replace('24:00', '00:00')

df['datetime'] = pd.to_datetime(df['datetime'])


In [ ]:
columns = [df[col].name for col in df.keys()]
columns

In [ ]:
def check_NA(dataframe: pd.DataFrame) -> None:
    NAs = [sum(df[col].isna()) for col in df[columns]]
    missing_values = zip(columns, NAs)
    print(dict([j for j in missing_values]))
    
check_NA(df)

In [6]:
vsel = df['state'].isna()
df.loc[vsel, 'state'] = 'None'

In [7]:
vsel = df['comments'].isna()
df.loc[vsel, 'comments'] = 'None'

In [8]:
vsel = df['shape'].isna()
df.loc[vsel, 'shape'] = 'None'

In [ ]:
df.dropna(axis=0, inplace=True)
check_NA(df)

In [14]:
df['year'] = df['datetime'].dt.year
country_year_counts = df.groupby(['country', 'year']).size().unstack(fill_value=0)

In [ ]:
# Plotten
country_year_counts.T.plot(kind='line', marker='')
plt.title('Datenpunkte pro Jahr und Stadt')
plt.xlabel('Jahr')
plt.ylabel('Anzahl der Datenpunkte')
plt.legend(title='Stadt')
plt.show()

In [24]:
from sklearn.cluster import DBSCAN
import pandas as pd
from geopy.distance import great_circle

# Beispiel für geografisches Clustering (DBSCAN)
coords = df[['latitude', 'longitude']].values
db = DBSCAN(eps=0.5, min_samples=2, metric=lambda x, y: great_circle(x, y).meters).fit(coords)
df['geo_cluster'] = db.labels_

In [64]:
UFOs = df[['latitude', 'longitude', 'geo_cluster']]
UFOs.head()

,latitude,longitude,geo_cluster
0,29.8830556,-97.941111,0
2,53.2,-2.916667,1
3,28.9783333,-96.645833,2
4,21.4180556,-157.803611,3
5,36.5950000,-82.188889,4


In [92]:
coords = []
for row, cols in UFOs.iterrows():
    coord = [float(UFOs.loc[row,'latitude']), float(UFOs.loc[row,'longitude']), UFOs.loc[row,'geo_cluster']]
    coords.append(coord)


In [95]:
UFOs.count()

latitude       70662
longitude      70662
geo_cluster    70662
dtype: int64

In [101]:
UFOs.groupby('geo_cluster').size()

geo_cluster
-1       6464
 0         21
 1          3
 2          3
 3         12
         ... 
 7923       2
 7924       2
 7925       2
 7926       2
 7927       2
Length: 7929, dtype: int64

In [65]:
import folium
import geopandas as gpd
from shapely.geometry import Point


# Startpunkt für die Karte
center = [30, -80]
zoom = 2
tiles = 'cartodb positron'

# Karte erstellen
cmap = folium.Map(location=center, zoom_start=zoom, tiles=tiles)

gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon, cluster in UFOs.iterrows()])

# Linie hinzufügen, die alle Punkte verbindet
for row, cols in UFOs.groupby(['country', 'year']).iterrows():
    folium.PolyLine(, weight=2.5, opacity=0.8).add_to(cmap)

# Marker mit Geschwindigkeit farblich codieren
for lat, lon, cluster in coords:
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Speed: {speed} km/h"
    ).add_to(cmap)

# Karte speichern
cmap


ValueError: too many values to unpack (expected 3)